In [5]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import re

# Load the data
topics_df = pd.read_csv('T3_categories.csv')
videos_df = pd.read_csv('T3-topics-per-vid.csv')
small_videos_df = pd.read_csv('T3_videos_per_topic.csv')

# Clean and prepare the data
def extract_topics(row):
    levels = []
    current_level = []
    for line in row['HLTA Topics'].split('\n'):
        if line.startswith('Level'):
            if current_level:
                levels.append(current_level)
                current_level = []
        elif line.strip() and not line.startswith('Video Title'):

            match = re.match(r'^(.*?):\s*\(([^)]+)\)\s*\(([^)]+)\)', line)
            if match:
                topic = match.group(1).strip()
                topic_id = match.group(2).strip()
                score = float(match.group(3).strip())
                current_level.append((topic, topic_id, score))
    if current_level:
        levels.append(current_level)
    return levels

videos_df['Topics'] = videos_df.apply(extract_topics, axis=1)


# Create a mapping from topic to videos
topic_to_videos = {}

for _, row in videos_df.iterrows():
    video_title = row['Video Title']
    link = row['Link']
    for level in row['Topics']:
        for topic, topic_id, score in level:  # now unpacking 3 values
            if topic not in topic_to_videos:
                topic_to_videos[topic] = []
            topic_to_videos[topic].append((video_title, link, score, topic_id))

# Create a mapping from video to topics
video_to_topics = {}

for _, row in videos_df.iterrows():
    video_title = row['Video Title']
    link = row['Link']
    topics = []
    
    for level_idx, level in enumerate(row['Topics'], 1):
        for topic, topic_id, score in level:  # Now unpacking topic_id
            # Match using the topic ID
            topic_details = topics_df[topics_df['Id'] == topic_id]
            if not topic_details.empty:
                general = topic_details.iloc[0]['General Category']
                specific = topic_details.iloc[0]['Specific Category']
            else:
                general = "Unknown"
                specific = "Unknown"
            
            topics.append({
                'topic': topic,
                'topic_id': topic_id,
                'level': level_idx,
                'score': score,
                'general_category': general,
                'specific_category': specific
            })
    
    video_to_topics[video_title] = {
        'link': link,
        'topics': topics
    }

# Create widgets
video_selector = widgets.Dropdown(
    options=[(title, title) for title in sorted(video_to_topics.keys())],
    description='Select Video:',
    disabled=False
)

topic_selector = widgets.Dropdown(
    options=[(topic, topic) for topic in sorted(topic_to_videos.keys())],
    description='Select Topic:',
    disabled=False
)

mode_selector = widgets.ToggleButtons(
    options=['Video Explorer', 'Topic Explorer'],
    description='Mode:',
    disabled=False,
    button_style=''
)

output = widgets.Output()

def display_video_info(video_title):
    with output:
        output.clear_output()
        video_info = video_to_topics[video_title]
        display(HTML(f"<h2>{video_title}</h2>"))
        display(HTML(f'<p><a href="{video_info["link"]}" target="_blank">Watch on YouTube</a></p>'))

        # Group topics by level
        levels = {}
        for topic in video_info['topics']:
            levels.setdefault(topic['level'], []).append(topic)

        for level in sorted(levels.keys()):
            display(HTML(f"<h3>Level {level} Topics</h3>"))
            table_html = """
                <table style='width:100%; border-collapse: collapse;' border='1'>
                    <tr>
                        <th>Topic</th>
                        <th>Score</th>
                        <th>General Category</th>
                        <th>Specific Category</th>
                    </tr>
            """
            for topic in sorted(levels[level], key=lambda x: x['score'], reverse=True):
                table_html += f"""
                    <tr>
                        <td>{topic['topic']}</td>
                        <td>{topic['score']:.2f}</td>
                        <td>{topic['general_category']}</td>
                        <td>{topic['specific_category']}</td>
                    </tr>
                """
            table_html += "</table>"
            display(HTML(table_html))

def display_topic_info(topic):
    with output:
        output.clear_output()
        display(HTML(f"<h2>Videos containing: {topic}</h2>"))

        # Get topic details
        topic_details = topics_df[topics_df['Texts'].str.contains(topic.split(',')[0].strip())]
        if not topic_details.empty:
            general = topic_details.iloc[0]['General Category']
            specific = topic_details.iloc[0]['Specific Category']
            display(HTML(f"<p><strong>General Category:</strong> {general}<br><strong>Specific Category:</strong> {specific}</p>"))

        videos = topic_to_videos[topic]
        table_html = """
            <table style='width:100%; border-collapse: collapse;' border='1'>
                <tr>
                    <th>Video Title</th>
                    <th>Score</th>
                    <th>Topic ID</th>
                    <th>Link</th>
                </tr>
        """
        for video_title, link, score, _ in sorted(videos, key=lambda x: x[2], reverse=True):
            table_html += f"""
                <tr>
                    <td>{video_title}</td>
                    <td>{score:.2f}</td>
                    <td><a href="{link}" target="_blank">Watch</a></td>
                </tr>
            """
        table_html += "</table>"
        display(HTML(table_html))


def on_mode_change(change):
    if change['new'] == 'Video Explorer':
        video_selector.layout.display = ''
        topic_selector.layout.display = 'none'
        display_video_info(video_selector.value)
    else:
        video_selector.layout.display = 'none'
        topic_selector.layout.display = ''
        display_topic_info(topic_selector.value)

def on_video_change(change):
    if mode_selector.value == 'Video Explorer':
        display_video_info(change['new'])

def on_topic_change(change):
    if mode_selector.value == 'Topic Explorer':
        display_topic_info(change['new'])

# Set up observers
mode_selector.observe(on_mode_change, names='value')
video_selector.observe(on_video_change, names='value')
topic_selector.observe(on_topic_change, names='value')

# Initial setup
video_selector.layout.display = ''
topic_selector.layout.display = 'none'

# Display the widgets
display(mode_selector)
display(video_selector)
display(topic_selector)
display(output)

# Initial display
display_video_info(video_selector.value)

ToggleButtons(description='Mode:', options=('Video Explorer', 'Topic Explorer'), value='Video Explorer')

Dropdown(description='Select Video:', layout=Layout(display=''), options=(('"10 RARE Animals Only Found in the…

Dropdown(description='Select Topic:', layout=Layout(display='none'), options=(('actual, key, holy', 'actual, k…

Output()

In [4]:
videos_df

,Video Title,Link,HLTA Topics,Topics
0,$200 Luxury Beach Hotel in The Philippines 🇵🇭,https://www.youtube.com/watch?v=--8n6A8Q6M0,"Level 3\nwoman, single, difference, opinion: (...","[[(woman, single, difference, opinion, Z1190, ..."
1,Marine reacts to the Philippine Light Reaction...,https://www.youtube.com/watch?v=-1B7cVoZr1c,"Level 3\nbusiness, issue: (Z1197) (1.00)\nhole...","[[(business, issue, Z1197, 1.0), (hole, rock, ..."
2,Ultimate Filipino Food Festival In The Netherl...,https://www.youtube.com/watch?v=-7vF5F-1btE,"Level 3\npork, meat, beef: (Z1131) (0.94)\nmea...","[[(pork, meat, beef, Z1131, 0.94), (meal, rest..."
3,SHOWING MY SISTER SB19 'GENTO' Music Video,https://www.youtube.com/watch?v=-9bfDHHneyU,"Level 3\nbusiness, issue: (Z1197) (1.00)\nperf...","[[(business, issue, Z1197, 1.0), (performance,..."
4,10 Reasons/Do not Retire TO the Philippines/Mo...,https://www.youtube.com/watch?v=-C5iB25BRsA,"Level 3\nwoman, single, difference, opinion: (...","[[(woman, single, difference, opinion, Z1190, ..."
...,...,...,...,...
2699,PHILIPPINES PROVINCE LIFE IS SO DIFFERENT FROM...,https://www.youtube.com/watch?v=zm_8N4vFnZw,"Level 3\nvacation, expat, cultural, simple, st...","[[(vacation, expat, cultural, simple, strong, ..."
2700,"Starring real-life gay couple, latest Filipino...",https://www.youtube.com/watch?v=zpT46etTP6E,"Level 3\nmother, father: (Z1168) (1.00)\nswimm...","[[(mother, father, Z1168, 1.0), (swimming, swi..."
2701,🇰🇷Koreans React to P-pop Idol Group | Lovey Do...,https://www.youtube.com/watch?v=zqvDHfgxWh8,"Level 3\nwoman, single, difference, opinion: (...","[[(woman, single, difference, opinion, Z1190, ..."
2702,FOREIGNER LIVING IN YHE PHILIPPINES WEEKLY FO...,https://www.youtube.com/watch?v=zvTP6wl9sTU,"Level 3\nchicken-chicken, hungry, table, finis...","[[(chicken-chicken, hungry, table, finish, Z15..."
